In [ ]:
# pip install --upgrade --no-deps --force-reinstall git+https://github.com/openai/whisper.git

In [11]:
#ffmpeg?????????????? ffmpeg-python lib?

In [14]:
# !pip install scipy
# ! pip install dtw-python

You should consider upgrading via the 'D:\programming\ML\univer\TEZA\env\Scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
import whisper
import torch
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd
from dtw import dtw
from scipy.ndimage import median_filter
import numpy as np

MODEL_WHISPER_SIZE = "small" #large
audio_file = "data/audio versiunea 1.mp3"

model = whisper.load_model(MODEL_WHISPER_SIZE, device="cpu")

def audio_to_text(audio):
    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    _, probs = model.detect_language(mel)

    detected_language = max(probs, key=probs.get)
    result = model.transcribe(audio)
    text = result["text"]
    return detected_language, text

d:\programming\ML\univer\TEZA\env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



100%|███████████████████████████████████████| 461M/461M [00:15<00:00, 31.1MiB/s]


In [2]:
# Importing the dtw module. When using in academic works please cite:
#   T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
#   J. Stat. Soft., doi:10.18637/jss.v031.i07.

In [3]:
# !pip uninstall torch -y
# !pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116

## Now need to get timestamps for each word

In [4]:
from whisper.tokenizer import get_tokenizer
import string 

AUDIO_SAMPLES_PER_TOKEN = whisper.audio.HOP_LENGTH * 2
AUDIO_TIME_PER_TOKEN = AUDIO_SAMPLES_PER_TOKEN / whisper.audio.SAMPLE_RATE
medfilt_width = 7
qk_scale = 1.0
languages = {"af_za": "Afrikaans", "am_et": "Amharic", "ar_eg": "Arabic", "as_in": "Assamese", "az_az": "Azerbaijani", "be_by": "Belarusian", "bg_bg": "Bulgarian", "bn_in": "Bengali", "bs_ba": "Bosnian", "ca_es": "Catalan", "cmn_hans_cn": "Chinese", "cs_cz": "Czech", "cy_gb": "Welsh", "da_dk": "Danish", "de_de": "German", "el_gr": "Greek", "en_us": "English", "es_419": "Spanish", "et_ee": "Estonian", "fa_ir": "Persian", "fi_fi": "Finnish", "fil_ph": "Tagalog", "fr_fr": "French", "gl_es": "Galician", "gu_in": "Gujarati", "ha_ng": "Hausa", "he_il": "Hebrew", "hi_in": "Hindi", "hr_hr": "Croatian", "hu_hu": "Hungarian", "hy_am": "Armenian", "id_id": "Indonesian", "is_is": "Icelandic", "it_it": "Italian", "ja_jp": "Japanese", "jv_id": "Javanese", "ka_ge": "Georgian", "kk_kz": "Kazakh", "km_kh": "Khmer", "kn_in": "Kannada", "ko_kr": "Korean", "lb_lu": "Luxembourgish", "ln_cd": "Lingala", "lo_la": "Lao", "lt_lt": "Lithuanian", "lv_lv": "Latvian", "mi_nz": "Maori", "mk_mk": "Macedonian", "ml_in": "Malayalam", "mn_mn": "Mongolian", "mr_in": "Marathi", "ms_my": "Malay", "mt_mt": "Maltese", "my_mm": "Myanmar", "nb_no": "Norwegian", "ne_np": "Nepali", "nl_nl": "Dutch", "oc_fr": "Occitan", "pa_in": "Punjabi", "pl_pl": "Polish", "ps_af": "Pashto", "pt_br": "Portuguese", "ro_ro": "Romanian", "ru_ru": "Russian", "sd_in": "Sindhi", "sk_sk": "Slovak", "sl_si": "Slovenian", "sn_zw": "Shona", "so_so": "Somali", "sr_rs": "Serbian", "sv_se": "Swedish", "sw_ke": "Swahili", "ta_in": "Tamil", "te_in": "Telugu", "tg_tj": "Tajik", "th_th": "Thai", "tr_tr": "Turkish", "uk_ua": "Ukrainian", "ur_pk": "Urdu", "uz_uz": "Uzbek", "vi_vn": "Vietnamese", "yo_ng": "Yoruba"}
selected_language = "en_us"
tokenizer = get_tokenizer(model.is_multilingual, language=languages[selected_language])

In [5]:
def split_tokens_on_unicode(tokens):
    words = []
    word_tokens = []
    current_tokens = []
    
    for token in tokens.tolist():
        current_tokens.append(token)
        decoded = tokenizer.decode_with_timestamps(current_tokens)
        if "\ufffd" not in decoded:
            words.append(decoded)
            word_tokens.append(current_tokens)
            current_tokens = []
    
    return words, word_tokens

In [6]:

def split_tokens_on_spaces(tokens):
    subwords, subword_tokens_list = split_tokens_on_unicode(tokens)
    words = []
    word_tokens = []
    
    for subword, subword_tokens in zip(subwords, subword_tokens_list):
        special = subword_tokens[0] >= tokenizer.eot
        with_space = subword.startswith(" ")
        punctuation = subword.strip() in string.punctuation
        if special or with_space or punctuation:
            words.append(subword)
            word_tokens.append(subword_tokens)
        else:
            words[-1] = words[-1] + subword
            word_tokens[-1].extend(subword_tokens)
    
    return words, word_tokens

In [7]:
if languages[selected_language] in {"Chinese", "Japanese", "Thai", "Lao", "Myanmar"}:
    # These languages don't typically use spaces, so it is difficult to split words
    # without morpheme analysis. Here, we instead split words at any
    # position where the tokens are decoded as valid unicode points
    split_tokens = split_tokens_on_unicode
else:
    split_tokens = split_tokens_on_spaces

In [8]:
# install hooks on the cross attention layers to retrieve the attention weights
QKs = [None] * model.dims.n_text_layer

for i, block in enumerate(model.decoder.blocks):
    block.cross_attn.register_forward_hook(
        lambda _, ins, outs, index=i: QKs.__setitem__(index, outs[-1])
    )

In [9]:
transcription= audio_to_text(audio_file)
transcription

d:\programming\ML\univer\TEZA\env\lib\site-packages\whisper\transcribe.py:77: UserWarning: Performing inference on CPU when CUDA is available
  warnings.warn("Performing inference on CPU when CUDA is available")
d:\programming\ML\univer\TEZA\env\lib\site-packages\whisper\transcribe.py:79: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


('ro',
 ' Personajul narrator din franceza Draza in tipologia conservatorului posturile din jurul sau se schimbă din ce cauza, de ce aici că trebuia să predea ca a sa unul mă simteam bine aici, eram singură, eram lăsat în pace, se întoarce ca să apărintească când era bine a s-a cu plăcere să am o cameră, le-am acaută caracterizare, pe personajului datorită negarea unul care nu putți spune despre mine că sunt prea gras prin autoevaluarea comportamentului său.')

In [19]:
# import os
# import io
# from scipy.io import wavfile
# import tarfile
# import urllib
# from tqdm import tqdm

# def download(url: str, target_path: str):
#     with urllib.request.urlopen(url) as source, open(target_path, "wb") as output:
#         with tqdm(total=int(source.info().get("Content-Length")), ncols=80, unit='iB', unit_scale=True, unit_divisor=1024) as loop:
#             while True:
#                 buffer = source.read(8192)
#                 if not buffer:
#                     break

#                 output.write(buffer)
#                 loop.update(len(buffer))

# class Fleurs(torch.utils.data.Dataset):
#     """
#     A simple class to wrap Fleurs and subsample a portion of the dataset as needed.
#     """
#     def __init__(self, lang, split="test", subsample_rate=1, device='cpu'):
#         url = f"https://storage.googleapis.com/xtreme_translations/FLEURS102/{lang}.tar.gz"
#         tar_path = os.path.expanduser(f"~/.cache/fleurs/{lang}.tgz")
#         os.makedirs(os.path.dirname(tar_path), exist_ok=True)

#         if not os.path.exists(tar_path):
#             download(url, tar_path)

#         all_audio = {}
#         with tarfile.open(tar_path, "r:gz") as tar:
#             for member in tar.getmembers():
#                 name = member.name
#                 if name.endswith(f"{split}.tsv"):
#                     labels = pd.read_table(tar.extractfile(member), names=("id", "file_name", "raw_transcription", "transcription", "_", "num_samples", "gender"))

#                 if f"/{split}/" in name and name.endswith(".wav"):
#                     audio_bytes = tar.extractfile(member).read()
#                     all_audio[os.path.basename(name)] = wavfile.read(io.BytesIO(audio_bytes))[1]                    

#         self.labels = labels.to_dict("records")[::subsample_rate]
#         self.all_audio = all_audio
#         self.device = device

#     def __len__(self):
#         return len(self.labels)

#     def __getitem__(self, item):
#         record = self.labels[item]
#         audio = torch.from_numpy(self.all_audio[record["file_name"]].copy())
#         text = record["transcription"]
        
#         return (audio, text)

# dataset = Fleurs(selected_language, subsample_rate=10)  # subsample 10% of the dataset for a quick demo


In [15]:
# for the first 10 examples in the dataset
# for (audio, label), transcription in zip(dataset, transcriptions[:10]):

audio = whisper.load_audio(audio_file)
duration = len(audio)
print(duration)
mel = whisper.log_mel_spectrogram(whisper.pad_or_trim(audio)).cpu()
tokens = torch.tensor(
    [
        *tokenizer.sot_sequence,
        tokenizer.timestamp_begin,
    ] + tokenizer.encode(transcription) + [
        tokenizer.timestamp_begin + duration // AUDIO_SAMPLES_PER_TOKEN,
        tokenizer.eot,
    ]
).cpu()
print(mel.size())
print(tokens.size())
with torch.no_grad():
    logits = model(mel.unsqueeze(0), tokens.unsqueeze(0))

weights = torch.cat(QKs)  # layers * heads * tokens * frames    
weights = weights[:, :, :, : duration // AUDIO_SAMPLES_PER_TOKEN].cpu()
weights = median_filter(weights, (1, 1, 1, medfilt_width))
weights = torch.tensor(weights * qk_scale).softmax(dim=-1)

w = weights / weights.norm(dim=-2, keepdim=True)
matrix = w[-6:].mean(axis=(0, 1))

alignment = dtw(-matrix.double().numpy())

jumps = np.pad(np.diff(alignment.index1s), (1, 0), constant_values=1).astype(bool)
jump_times = alignment.index2s[jumps] * AUDIO_TIME_PER_TOKEN
words, word_tokens = split_tokens(tokens)

# display the normalized attention weights and the alignment
plt.figure(figsize=(8, 8))
plt.imshow(matrix, aspect="auto")
plt.plot(alignment.index2s, alignment.index1s, color="red")

xticks = np.arange(0, matrix.shape[1], 1 / AUDIO_TIME_PER_TOKEN)
xticklabels = (xticks * AUDIO_TIME_PER_TOKEN).round().astype(np.int32) 
plt.xticks(xticks, xticklabels)
plt.xlabel("Time (s)")

# display tokens and words as tick labels
ylims = plt.gca().get_ylim()

ax = plt.gca()
ax.tick_params('both', length=0, width=0, which='minor', pad=6)

ax.yaxis.set_ticks_position("left")
ax.yaxis.set_label_position("left")
ax.invert_yaxis()
ax.set_ylim(ylims)

major_ticks = [-0.5]
minor_ticks = []
current_y = 0

for word, word_token in zip(words, word_tokens):
    minor_ticks.append(current_y + len(word_token) / 2 - 0.5)
    current_y += len(word_token)
    major_ticks.append(current_y - 0.5)
    
ax.yaxis.set_minor_locator(ticker.FixedLocator(minor_ticks))
ax.yaxis.set_minor_formatter(ticker.FixedFormatter(words))
ax.set_yticks(major_ticks)
ax.yaxis.set_major_formatter(ticker.NullFormatter())

# for label in ax.get_yminorticklabels():
#     label.set_fontproperties(prop)

plt.ylabel("Words")
plt.show()

# display the word-level timestamps in a table
word_boundaries = np.pad(np.cumsum([len(t) for t in word_tokens[:-1]]), (1, 0))
begin_times = jump_times[word_boundaries[:-1]]
end_times = jump_times[word_boundaries[1:]]

data = [
    dict(word=word, begin=begin, end=end)
    for word, begin, end in zip(words[:-1], begin_times, end_times)
    if not word.startswith("<|") and word.strip() not in ".,!?、。"
]

display(pd.DataFrame(data))
display(HTML("<hr>"))

1048660
torch.Size([80, 3000])
torch.Size([164])


IndexError: index out of range in self

In [13]:
import torch
print(torch.cuda.is_available())

False
